In [15]:
import pandas as pd
import re
from unidecode import unidecode

In [16]:
chunk_size = 1000000
chunks = pd.read_csv(r"D:\tesis_u_sabana\data\base_procesos.csv", encoding='utf-8', chunksize=chunk_size,sep='~')
df_procesos = pd.concat(chunks)

C:\Users\pipe\AppData\Local\Temp\ipykernel_9144\1598050671.py:3: DtypeWarning: Columns (1,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesos = pd.concat(chunks)
C:\Users\pipe\AppData\Local\Temp\ipykernel_9144\1598050671.py:3: DtypeWarning: Columns (1,8,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesos = pd.concat(chunks)
C:\Users\pipe\AppData\Local\Temp\ipykernel_9144\1598050671.py:3: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesos = pd.concat(chunks)
C:\Users\pipe\AppData\Local\Temp\ipykernel_9144\1598050671.py:3: DtypeWarning: Columns (1,8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesos = pd.concat(chunks)
C:\Users\pipe\AppData\Local\Temp\ipykernel_9144\1598050671.py:3: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesos

In [11]:
df_procesos.head()

entidad nit_entidad  \
0                   E.S.E HOSPITAL DIOGENES TRONCOSO   860024030   
1  UNIDAD ADMINISTRATIVA ESPECIAL DE SALUD DE ARAUCA   900034608   
2  HOSPITAL FEDERICO LLERAS ACOSTA DE IBAGUE TOLI...   890706833   
3                  ALCALDÍA DISTRITAL DE SANTA MARTA   891780009   
4                                IGAC Sede Central *   899999004   

         departamento_entidad ciudad_entidad ordenentidad  \
0                Cundinamarca  Puerto Salgar  Territorial   
1                      Arauca    No Definido  Territorial   
2                      Tolima         Ibagué  Territorial   
3                   Magdalena    Santa Marta  Territorial   
4  Distrito Capital de Bogotá    No Definido     Nacional   

  entidad_centralizada   id_del_proceso                referencia_del_proceso  \
0      Descentralizada  CO1.REQ.7712925                           CO-054-2025   
1         Centralizada  CO1.REQ.3408112  CM-003-2022 (Presentación de oferta)   
2      Descentralizada  CO1.REQ.3603732                          COM-164-2022   
3         Centralizada  CO1.REQ.1970309                            PS520-2021   
4         Centralizada  CO1.REQ.3511740                      CD-CAL-1037-2022   

         pci id_del_portafolio  ... nombre_del_proveedor_adjudicado  \
0  704063031  CO1.BDOS.7578121  ...                     No Definido   
1  700587157  CO1.BDOS.3277220  ...        LRAMIREZ CONSULTORES SAS   
2  700665045  CO1.BDOS.3511066  ...                     No Definido   
3  702354598  CO1.BDOS.1917245  ...     ALBERTO LUIS ANDRADE MERIÑO   
4  700663107  CO1.BDOS.3421207  ...      EDWARD NARCISO ROJAS OLAYA   

  nit_del_proveedor_adjudicado codigo_principal_de_categoria  \
0                  No Definido                   V1.80111600   
1                    900843788                   V1.80101500   
2                  No Definido                   V1.44103103   
3                   1083029933                   V1.80111600   
4                      9729329                   V1.80161501   

  estado_de_apertura_del_proceso         tipo_de_contrato subtipo_de_contrato  \
0                        Abierto      Decreto 092 de 2017         No Definido   
1                        Cerrado            Interventoría         No Definido   
2                        Abierto              Suministros         No Definido   
3                        Abierto  Prestación de servicios         No Definido   
4                        Abierto  Prestación de servicios         No Definido   

  categorias_adicionales                                         urlproceso  \
0            No definido  https://community.secop.gov.co/Public/Tenderin...   
1             V180101600  https://community.secop.gov.co/Public/Tenderin...   
2            No definido  https://community.secop.gov.co/STS/Users/Login...   
3             V180111701  https://community.secop.gov.co/Public/Tenderin...   
4            No definido  https://community.secop.gov.co/Public/Tenderin...   

  codigo_entidad          estado_resumen  
0      704063031  Presentación de oferta  
1      700587157              Adjudicado  
2      700665045             No Definido  
3      702354598              Adjudicado  
4      700663107              Adjudicado  

[5 rows x 59 columns]

In [17]:
df_procesos.columns

Index(['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad',
       'ordenentidad', 'entidad_centralizada', 'id_del_proceso',
       'referencia_del_proceso', 'pci', 'id_del_portafolio',
       'nombre_del_procedimiento', 'descripción_del_procedimiento', 'fase',
       'fecha_de_publicacion_del_proceso', 'fecha_de_ultima_publicación',
       'fecha_de_publicacion_fase_planeacion_precalificacion',
       'fecha_de_publicacion_fase_seleccion_precalificacion',
       'fecha_de_publicacion_manifestacion_de_interes',
       'fecha_de_publicacion_fase_borrador',
       'fecha_de_publicacion_fase_seleccion', 'precio_base',
       'modalidad_de_contratacion', 'justificación_modalidad_de_contratación',
       'duracion', 'unidad_de_duracion', 'fecha_de_recepcion_de_respuestas',
       'fecha_de_apertura_de_respuesta', 'fecha_de_apertura_efectiva',
       'ciudad_de_la_unidad_de_contratación',
       'nombre_de_la_unidad_de_contratación', 'proveedores_invitados',
       'proveedore

In [18]:
column_names= df_procesos .columns

def clean_column_name(col_name):
    replacements = {
        " ": "_",
        "(": "",
        ")": "",
        "-": "_",
        "ID ": "",
        "Fecha de ": "f_",
        "Publicacion": "pub",
        "Procedimiento": "proc",
        "Proveedor": "prov",
        "Contratacion": "contr",
        "Adjudicacion": "adj"
    }
    col_name = col_name.lower()  # Convertir a minúsculas
    for key, value in replacements.items():
        col_name = col_name.replace(key, value)
    return col_name

cleaned_column_names = [clean_column_name(col) for col in column_names]

df_procesos.columns = cleaned_column_names

In [ ]:
# Cargar el DataFrame original
df = df_procesos.copy()

# Step 1: Eliminar saltos de línea
df = df.replace('\n', ' ', regex=True)

# Step 2: Reemplazar delimitadores conflictivos (cambiar comas por punto y coma)
df = df.replace(',', ';', regex=True)

# Step 3: Eliminar caracteres no imprimibles
df = df.replace(r'[\x00-\x1F\x7F]', '', regex=True)

# Step 4: Normalizar textos (eliminar tildes y caracteres especiales)
def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = unidecode(texto)  # Elimina tildes y caracteres especiales
        texto = re.sub(r'[^a-zA-Z0-9\s.,;:_-]', '', texto)  # Mantiene solo caracteres válidos
        texto = texto.strip()  # Elimina espacios extra al inicio y final
    return texto

df = df.applymap(limpiar_texto)

# Step 5: Convertir columnas con datos mixtos a string
for col in df.columns:
    if df[col].apply(type).nunique() > 1:
        df[col] = df[col].astype(str)

# Step 6: Eliminar duplicados
df = df.drop_duplicates()

In [21]:
df.to_csv(r"D:\tesis_u_sabana\data\secop_2_procesos_1.csv", index=False, sep='~')

In [19]:
df[0:10000].to_csv(r"D:\tesis_u_sabana\data\secop_2_procesos_1_tiny.csv", index=False, sep='~')

In [20]:
df.columns

Index(['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad',
       'ordenentidad', 'entidad_centralizada', 'id_del_proceso',
       'referencia_del_proceso', 'pci', 'id_del_portafolio',
       'nombre_del_procedimiento', 'descripción_del_procedimiento', 'fase',
       'fecha_de_publicacion_del_proceso', 'fecha_de_ultima_publicación',
       'fecha_de_publicacion_fase_planeacion_precalificacion',
       'fecha_de_publicacion_fase_seleccion_precalificacion',
       'fecha_de_publicacion_manifestacion_de_interes',
       'fecha_de_publicacion_fase_borrador',
       'fecha_de_publicacion_fase_seleccion', 'precio_base',
       'modalidad_de_contratacion', 'justificación_modalidad_de_contratación',
       'duracion', 'unidad_de_duracion', 'fecha_de_recepcion_de_respuestas',
       'fecha_de_apertura_de_respuesta', 'fecha_de_apertura_efectiva',
       'ciudad_de_la_unidad_de_contratación',
       'nombre_de_la_unidad_de_contratación', 'proveedores_invitados',
       'proveedore

In [22]:
df['departamento_entidad'].unique()

array(['Cundinamarca', 'Arauca', 'Tolima', 'Magdalena',
       'Distrito Capital de Bogotá', 'Chocó', 'Nariño', 'Huila',
       'Bolívar', 'Antioquia', 'Quindío', 'Cesar', 'Valle del Cauca',
       'Boyacá', 'Atlántico', 'Cauca', 'Risaralda', 'Santander',
       'Caquetá', 'Caldas', 'La Guajira', 'Casanare', 'Meta', 'Sucre',
       'Córdoba', 'Putumayo', 'Guaviare', 'Norte de Santander',
       'San Andrés; Providencia y Santa Catalina', 'No Definido',
       'Amazonas', 'Vichada', 'Vaupés', 'Guainía'], dtype=object)